In [1]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

import h1st.core as h1
h1.init()

from examples.RayTune.sklearn_smv_classifier import SklearnSVMClassifier

from examples.RayTune import config
from examples.RayTune.utils import prepare_train_test_data

In [3]:
import ray
from ray import tune

ray.init(ignore_reinit_error=True)

2020-11-02 10:23:16,806	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.2',
 'raylet_ip_address': '192.168.1.2',
 'redis_address': '192.168.1.2:6379',
 'object_store_address': '/tmp/ray/session_2020-11-02_10-23-16_329246_31370/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-02_10-23-16_329246_31370/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-11-02_10-23-16_329246_31370',
 'metrics_export_port': 65330}

## Class H1stTune

In [4]:
import json
import os    

from filelock import FileLock 

from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier

# class H1stTune(tune.Trainable):

#     h1st_ml_model = None
#     parameters = {}
    
#     def setup(self, config):
#         from filelock import FileLock 
#         self.config = config
#         self.timestep = 0

model_class = TensorflowMLPClassifier

class TrainH1stModel(tune.Trainable):    
    def setup(self, config):
        self.timestep = 0
#         self.config = config
#         self.timestep = 0
        self.kwargs = {}
        self.hyperparameter = getattr(model_class, 'hyperparameter', {})['tuning_param']
        for k, v in self.hyperparameter.items():
            self.kwargs[k] = v(config[k])
#         kwargs = {
#             'units': int(config['units']),
#             'epochs': int(config['epochs']), 
#             'lr': config['lr'], 
#             'n_layer': int(config['n_layer'])
#         }

        self.h1_ml_model = model_class(**self.kwargs)
#         self.h1_ml_model = model_class(
#             units=int(config['units']), 
#             epochs=int(config['epochs']), 
#             lr=config['lr'], 
#             n_layer=int(config['n_layer']))

#     TensorflowMLPClassifier(
#             units=int(config['units']), 
#             epochs=int(config['epochs']), 
#             lr=config['lr'], 
#             n_layer=int(config['n_layer']))
        
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_ml_model.load_data()
        self.prepared_data = self.h1_ml_model.prep(data)  
        
    def step(self):
        self.timestep += 1
        self.h1_ml_model.train(self.prepared_data)
        self.h1_ml_model.evaluate(self.prepared_data)
        acc = self.h1_ml_model.metrics['accuracy']
        self.h1_ml_model.persist(str(self.timestep) + str())
        return {"mean_accuracy": acc}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]



hyperparameter = {
    'tuning_param': {
        'lr': float, # 0.01 ~ 0.1
        'units': int # 4 ~ 16
    },
    'target_metrics': {
        'accuracy': 'minimize'
    }
}

            
class TrainH1stModel2(tune.Trainable):
#     def __init__(self, h1st_model):
#         self.h1_tf_mlp = h1st_model
    def setup(self, config):
        self.timestep = 0
        self.config = config
        self.h1_tf_mlp = TensorflowMLPClassifier(
            units=int(config['units']), 
            epochs=int(config['epochs']), 
            lr=config['lr'], 
            n_layer=int(config['n_layer']))
        
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)  
        
    def step(self):
        self.timestep += 1
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)
        acc = self.h1_tf_mlp.metrics['accuracy']
        self.h1_tf_mlp.persist(str(self.timestep) + str())
        return {"accuracy": acc}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]
            
            
class MyTrainableClass(tune.Trainable):
    """Example agent whose learning curve is a random sigmoid.

    The dummy hyperparameters "width" and "height" determine the slope and
    maximum reward value reached.
    """

    def setup(self, config):
        self.timestep = 0

    def step(self):
        self.timestep += 1
        v = np.tanh(float(self.timestep) / self.config.get("width", 1))
        v *= self.config.get("height", 1)

        # Here we use `episode_reward_mean`, but you can also report other
        # objectives such as loss or accuracy.
        return {"episode_reward_mean": v}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def load_checkpoint(self, checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]            

In [ ]:
tmp_model = TensorflowMLPClassifier(
            units=32, 
            epochs=5, 
            lr=0.01, 
            n_layer=5)

In [ ]:
tmp_model.hyperparameter

## Population Base Training

In [ ]:
# config = {
#     "lr": tune.grid_search([0.005]),
#     "units": tune.grid_search([16, 32, 64, 128]),
#     "n_layer": tune.grid_search([3, 5, 10])
# #     "n_layer": tune.uniform(1, 10)
# }

# analysis = tune.run(
#     TrainH1stModel, 
#     config=config, 
#     verbose=2,
#     stop={"training_iteration": 3}
# )

In [27]:
from ray.tune.schedulers import PopulationBasedTraining

# experiment_metrics = dict(metric="accuracy", mode="max")

pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        perturbation_interval=2,  # Every N time_attr units, "perturb" the parameters.
        hyperparam_mutations={           
#             "lr": [0.005],
            "lr": lambda: random.uniform(0.001, 0.01),
            "epochs": [2, 10, 30, 100],
            "units": [4, 8, 16, 32, 64],
            "n_layer": [2, 4, 6]            
        })

# Note: This appears to be needed to avoid a "key error", but in fact these values won't change
# in the analysis.dataframe() object, even though they will be tuned by the PBT scheduler.
# So when you look at the analysis.dataframe(), look at the `experiment_tag` to see the actual values!
config = {
    "lr": 0.005,            # Use the lowest values from the previous definition
    "units": 16,
    "n_layer": 4,
    "epochs": 2
}

In [28]:
# analysis = tune.run(TrainH1stModel, 
#     scheduler=pbt_scheduler, 
#     config=config,
#     stop={"training_iteration": 4},
# #     stop={"mean_accuracy": 0.82, "training_iteration": 200},                    
#     num_samples=40,
#     verbose=1,
#     metric="mean_accuracy",
#     mode="max"
# )b

analysis= tune.run(
    TrainH1stModel2,
    config=config,
    metric="accuracy",
    mode="max",
    scheduler=pbt_scheduler,
    stop={"training_iteration": 6},
    verbose=2,
    num_samples=20
)

Trial name,status,loc,iter,total time (s),accuracy
TrainH1stModel2_14f21_00000,TERMINATED,,10,8.87063,0.816465
TrainH1stModel2_14f21_00001,ERROR,,3,2.79469,0.799495
TrainH1stModel2_14f21_00002,TERMINATED,,10,7.73855,0.819495
TrainH1stModel2_14f21_00003,TERMINATED,,10,9.23857,0.817475
TrainH1stModel2_14f21_00004,ERROR,,1,1.1864,0.803333
TrainH1stModel2_14f21_00005,TERMINATED,,10,8.12699,0.818485
TrainH1stModel2_14f21_00006,TERMINATED,,10,9.03345,0.817879
TrainH1stModel2_14f21_00007,TERMINATED,,10,7.31378,0.817475
TrainH1stModel2_14f21_00008,TERMINATED,,10,7.88928,0.818384
TrainH1stModel2_14f21_00009,ERROR,,3,2.80409,0.800404


TuneError: ('Trials did not complete', [TrainH1stModel2_14f21_00001, TrainH1stModel2_14f21_00004, TrainH1stModel2_14f21_00009, TrainH1stModel2_14f21_00013, TrainH1stModel2_14f21_00014, TrainH1stModel2_14f21_00015])

AttributeError: 'dict' object has no attribute 'set'

In [ ]:
analysis.dataframe()[['accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('accuracy', ascending=False).head(20)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## BO Hyperband (BOHB)

In [ ]:
# import json
# import os

# import numpy as np

# import ray
# from ray import tune
# from ray.tune import Trainable
# from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
# from ray.tune.suggest.bohb import TuneBOHB


# class MyTrainableClass(Trainable):
#     """Example agent whose learning curve is a random sigmoid.

#     The dummy hyperparameters "width" and "height" determine the slope and
#     maximum reward value reached.
#     """

#     def setup(self, config):
#         self.timestep = 0

#     def step(self):
#         self.timestep += 1
#         v = np.tanh(float(self.timestep) / self.config.get("width", 1))
#         v *= self.config.get("height", 1)

#         # Here we use `episode_reward_mean`, but you can also report other
#         # objectives such as loss or accuracy.
#         return {"episode_reward_mean": v}

#     def save_checkpoint(self, checkpoint_dir):
#         path = os.path.join(checkpoint_dir, "checkpoint")
#         with open(path, "w") as f:
#             f.write(json.dumps({"timestep": self.timestep}))
#         return path

#     def load_checkpoint(self, checkpoint_path):
#         with open(checkpoint_path) as f:
#             self.timestep = json.loads(f.read())["timestep"]


# if __name__ == "__main__":
#     import ConfigSpace as CS  # noqa: F401
#     ray.init(num_cpus=8)

#     config = {
#         "iterations": 100,
#         "width": tune.uniform(0, 20),
#         "height": tune.uniform(-100, 100),
#         "activation": tune.choice(["relu", "tanh"])
#     }

#     # Optional: Pass the parameter space yourself
#     # config_space = CS.ConfigurationSpace()
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("width", lower=0, upper=20))
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("height", lower=-100, upper=100))
#     # config_space.add_hyperparameter(
#     #     CS.CategoricalHyperparameter(
#     #         "activation", choices=["relu", "tanh"]))

#     experiment_metrics = dict(metric="episode_reward_mean", mode="max")

#     bohb_hyperband = HyperBandForBOHB(
#         time_attr="training_iteration",
#         max_t=100,
#         reduction_factor=4,
#         **experiment_metrics)

#     bohb_search = TuneBOHB(
#         # space=config_space,  # If you want to set the space manually
#         max_concurrent=4,
#         **experiment_metrics)

#     tune.run(
#         MyTrainableClass,
#         name="bohb_test",
#         config=config,
#         scheduler=bohb_hyperband,
#         search_alg=bohb_search,
#         num_samples=10,
#         stop={"training_iteration": 100})

In [5]:
import ConfigSpace as CS
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

In [ ]:
config_space = CS.ConfigurationSpace()

# There are also UniformIntegerHyperparameter and UniformFloatHyperparameter
# objects for defining integer and float ranges, respectively. For example:
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('foo', lower=0, upper=100))

config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter('lr', lower=0.005, upper=0.05))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('units', lower=1, upper=64))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('n_layer', lower=1, upper=8))
config_space.add_hyperparameter(
    CS.UniformIntegerHyperparameter('epochs', lower=4, upper=50))


# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('lr', choices=[0.005]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('units', choices=[4, 8, 16, 32]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('n_layer', choices=[2, 4, 6, 8]))
# config_space.add_hyperparameter(
#     CS.CategoricalHyperparameter('epochs', choices=[2, 10, 30, 100]))

In [ ]:
# tune.run(
#     MyTrainableClass,
#     name="bohb_test",
#     config=config,
#     scheduler=bohb_hyperband,
#     search_alg=bohb_search,
#     num_samples=10,
#     stop={"training_iteration": 100})

In [6]:
config = {
    "lr": tune.uniform(0.002, 0.01),
    "units": tune.uniform(4, 32),
    "n_layer": tune.uniform(1, 8),    
    "epochs": tune.choice([10, 5])
}

In [14]:
experiment_metrics = dict(metric="accuracy", mode="max")
algo = TuneBOHB(
#     config_space, 
    max_concurrent=10, 
    **experiment_metrics
)
# algo = TuneBOHB(config_space, max_concurrent=10)
# search_algorithm = TuneBOHB(config_space, max_concurrent=4)

scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    **experiment_metrics
)

In [15]:
analysis= tune.run(
    TrainH1stModel2,
    config=config,
#     metric="mean_accuracy",
#     mode="max",     
    search_alg=algo,
    scheduler=scheduler,
    stop={"training_iteration": 5},
    verbose=2,
    num_samples=20
)

# analysis = tune.run(
#     TrainH1stModel,
#     config=config,
#     scheduler=bohb_hyperband,
#     search_alg=bohb_search,
#     num_samples=10,
#     stop={"training_iteration": 100})

# analysis = tune.run(
#     TrainH1stModel, 
#     scheduler=scheduler, 
#     search_alg=search_algorithm, 
#     metric="mean_accuracy",
#     mode="max",         
#     num_samples=40,                           # Force it try all 12 combinations
#     stop={"training_iteration": 4},
#     verbose=1
# )

Trial name,status,loc,epochs,lr,n_layer,units
TrainH1stModel2_1a9fff06,RUNNING,,5,0.00501792,5.10137,11.1177
TrainH1stModel2_1aa03160,PENDING,,5,0.00676194,7.63019,13.1718
TrainH1stModel2_1aa05e9c,PENDING,,5,0.00403044,4.34325,4.26195
TrainH1stModel2_1aa0893a,PENDING,,5,0.00700041,5.57598,24.6386
TrainH1stModel2_1aa0b9b4,PENDING,,10,0.00386175,2.84424,23.22
TrainH1stModel2_1aa0e74a,PENDING,,5,0.0072658,3.30759,22.9984
TrainH1stModel2_1aa11396,PENDING,,10,0.00924774,6.56662,22.8909
TrainH1stModel2_1aa13fec,PENDING,,5,0.00330441,4.75389,12.8086
TrainH1stModel2_1aa171a6,PENDING,,5,0.00973301,6.8499,8.49629
TrainH1stModel2_1aa1a950,PENDING,,10,0.00806005,5.5282,16.7041


(pid=31629) 2020-11-02 10:42:09.497557: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=31629) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=31629) 2020-11-02 10:42:09.513004: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f976e5f0a80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
(pid=31629) 2020-11-02 10:42:09.513038: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
(pid=31645) 2020-11-02 10:42:09.520179: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=31645) To enable them i

(pid=31646) INFO:filelock:Lock 10652474512 released on ./data/credit_card/data.lock
(pid=31646) INFO:examples.RayTune.utils:(20100, 23), (9900, 23), (20100, 1), (9900, 1)
(pid=31646) 2020-11-02 10:42:23,604	INFO trainable.py:255 -- Trainable.setup took 14.098 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=31629) INFO:filelock:Lock 10624469968 released on ./data/credit_card/data.lock
(pid=31629) INFO:examples.RayTune.utils:(20100, 23), (9900, 23), (20100, 1), (9900, 1)
(pid=31627) INFO:filelock:Lock 10603221200 released on ./data/credit_card/data.lock
(pid=31627) INFO:examples.RayTune.utils:(20100, 23), (9900, 23), (20100, 1), (9900, 1)
(pid=31627) 2020-11-02 10:42:24,145	INFO trainable.py:255 -- Trainable.setup took 14.648 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=31629) 2020-11-02 10:42:24,148	INFO trainable.py:255 -- Traina

Result for TrainH1stModel2_1aa05e9c:
  accuracy: 0.806969696969697
  date: 2020-11-02_10-42-24
  done: false
  experiment_id: 049baf58b6d5470eb314255b783dc9d9
  experiment_tag: 3_epochs=5,lr=0.0040304,n_layer=4.3433,units=4.262
  hostname: arimos-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 192.168.1.2
  pid: 31646
  time_since_restore: 0.9162938594818115
  time_this_iter_s: 0.9162938594818115
  time_total_s: 0.9162938594818115
  timestamp: 1604342544
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1aa05e9c
  


Trial name,status,loc,epochs,lr,n_layer,units,iter,total time (s),accuracy
TrainH1stModel2_1a9fff06,RUNNING,,5,0.00501792,5.10137,11.1177,,,
TrainH1stModel2_1aa03160,RUNNING,,5,0.00676194,7.63019,13.1718,,,
TrainH1stModel2_1aa05e9c,PAUSED,,5,0.00403044,4.34325,4.26195,1,0.916294,0.80697
TrainH1stModel2_1aa0893a,RUNNING,,5,0.00700041,5.57598,24.6386,,,
TrainH1stModel2_1aa0b9b4,RUNNING,,10,0.00386175,2.84424,23.22,,,
TrainH1stModel2_1aa0e74a,RUNNING,,5,0.0072658,3.30759,22.9984,,,
TrainH1stModel2_1aa11396,RUNNING,,10,0.00924774,6.56662,22.8909,,,
TrainH1stModel2_1aa13fec,RUNNING,,5,0.00330441,4.75389,12.8086,,,
TrainH1stModel2_1aa171a6,RUNNING,,5,0.00973301,6.8499,8.49629,,,
TrainH1stModel2_1aa1a950,RUNNING,,10,0.00806005,5.5282,16.7041,,,


...

(pid=31915) INFO:examples.RayTune.tensorflow_mlp_classifier:(9900, 1), (9900, 1)
(pid=31915) 2020-11-02 10:43:53,291 INFO h1st.model_repository.model_repository: Saving metrics property...
(pid=31915) INFO:h1st.model_repository.model_repository:Saving metrics property...
(pid=31915) 2020-11-02 10:43:53,292 INFO h1st.model_repository.model_repository: Saving stats property...
(pid=31915) INFO:h1st.model_repository.model_repository:Saving stats property...
(pid=31915) 2020-11-02 10:43:53,293 INFO h1st.model_repository.model_repository: Saving model property...
(pid=31915) INFO:h1st.model_repository.model_repository:Saving model property...


Result for TrainH1stModel2_26f63518:
  accuracy: 0.8109090909090909
  date: 2020-11-02_10-43-53
  done: true
  experiment_id: 42abd4c40cf9491bae93cf3d3b9bcd5f
  experiment_tag: 14_epochs=5,lr=0.0074168,n_layer=7.5272,units=17.104
  hostname: arimos-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 192.168.1.2
  pid: 31914
  time_since_restore: 1.0933690071105957
  time_this_iter_s: 1.0933690071105957
  time_total_s: 7.726242780685425
  timestamp: 1604342633
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: 26f63518
  


(pid=31914) INFO:examples.RayTune.tensorflow_mlp_classifier:(9900, 1), (9900, 1)
(pid=31914) 2020-11-02 10:43:53,527 INFO h1st.model_repository.model_repository: Saving metrics property...
(pid=31914) INFO:h1st.model_repository.model_repository:Saving metrics property...
(pid=31914) 2020-11-02 10:43:53,528 INFO h1st.model_repository.model_repository: Saving stats property...
(pid=31914) INFO:h1st.model_repository.model_repository:Saving stats property...
(pid=31914) 2020-11-02 10:43:53,528 INFO h1st.model_repository.model_repository: Saving model property...
(pid=31914) INFO:h1st.model_repository.model_repository:Saving model property...


Trial name,status,loc,epochs,lr,n_layer,units,iter,total time (s),accuracy
TrainH1stModel2_1a9fff06,TERMINATED,,5,0.00501792,5.10137,11.1177,4,7.08793,0.816465
TrainH1stModel2_1aa03160,TERMINATED,,5,0.00676194,7.63019,13.1718,4,6.86255,0.816869
TrainH1stModel2_1aa05e9c,TERMINATED,,5,0.00403044,4.34325,4.26195,4,5.54822,0.812626
TrainH1stModel2_1aa0893a,TERMINATED,,5,0.00700041,5.57598,24.6386,5,7.64614,0.815051
TrainH1stModel2_1aa0b9b4,TERMINATED,,10,0.00386175,2.84424,23.22,4,5.46405,0.816263
TrainH1stModel2_1aa0e74a,TERMINATED,,5,0.0072658,3.30759,22.9984,4,5.7722,0.817071
TrainH1stModel2_1aa11396,TERMINATED,,10,0.00924774,6.56662,22.8909,5,7.79449,0.816768
TrainH1stModel2_1aa13fec,TERMINATED,,5,0.00330441,4.75389,12.8086,1,1.14893,0.801919
TrainH1stModel2_1aa171a6,TERMINATED,,5,0.00973301,6.8499,8.49629,4,7.00579,0.81798
TrainH1stModel2_1aa1a950,TERMINATED,,10,0.00806005,5.5282,16.7041,4,7.09906,0.816768


TuneError: ('Trials did not complete', [TrainH1stModel2_2f6d7fda, TrainH1stModel2_2f7fc38e])

In [16]:
analysis.dataframe()[['accuracy', 'config/epochs','config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

NameError: name 'analysis' is not defined

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## Bayesian Optimization (BO)

In [ ]:
"""This test checks that BayesOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import time


import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


# def evaluation_fn(step, width, height):
#     return (0.1 + width * step / 100)**(-1) + height * 0.1


# def easy_objective(config):
#     # Hyperparameters
#     width, height = config["width"], config["height"]

#     for step in range(config["steps"]):
#         # Iterative training function - can be any arbitrary training procedure
#         intermediate_score = evaluation_fn(step, width, height)
#         # Feed the score back back to Tune.
#         tune.report(iterations=step, mean_loss=intermediate_score)
#         time.sleep(0.1)

space = {
    "lr": tune.uniform(0.005, 0.05),
    "units": tune.uniform(1, 64),
    "epochs": tune.uniform(4, 50),    
#     "units": tune.randint(1, 64),
#     "n_layer": tune.randint(1, 8)
    "n_layer": tune.quniform(1, 8, 1.0),    
}

algo = BayesOptSearch(   
    utility_kwargs={
    "kind": "ucb",
    "kappa": 2.5,
    "xi": 0.0
}
)

algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

analysis= tune.run(
    TrainH1stModel,
    config=space,
    metric="mean_accuracy",
    mode="max",     
#     name="my_exp",
    search_alg=algo,
#     scheduler=scheduler,
    stop={"training_iteration": 5},
    verbose=1,
    num_samples=200
)


In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

### with AsyncHyperBandScheduler

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## OptunaSearch

In [ ]:
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch

In [ ]:
tune_kwargs = {
    "num_samples": 200,
    "config": {
        "lr": tune.uniform(0.005, 0.05),
        "units": tune.uniform(1, 64),
        "epochs": tune.uniform(4, 50),    
        "n_layer": tune.choice([2, 4, 6]),           
    }
}
algo = OptunaSearch()
algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
    TrainH1stModel2,
    metric="mean_accuracy",
    mode="max",
    search_alg=algo,
    scheduler=scheduler,
    **tune_kwargs)

In [ ]:
analysis.dataframe().head(10)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

## HyperOptSearch